#  Comparativo de solver basado en multiplicadores de Lagrange y método de Newton

Para comparar el desempeño de ambos solvers, se decidió variar el valor del rendimiento $r$ en un rango de 0.4 a 1. Al respecto, los resultados obtenidos se resumen en la siguiente tabla:

[WIP: añadir tabla que compare la norma de la diferencia y el valor absoluto entre ambos]

De lo anterior, se desprenden los siguientes hallazgos:

* La diferencia entre las soluciones aumenta conforme aumenta el rendimiento deseado.

* A pesar del aumento en la diferencia, ambas soluciones cumplen con las restricciones deseadas.

* El riesgo con el solver usando los multiplicadores de Lagrange siempre es igual o menor que el del solver con el método de Newton. 

* La solución es mejor usando el solver basado en los multiplicadores de Lagrange. 




## Librerías

In [0]:
import numpy as np
import pandas as pd
import cupy as cp
import solver.extraer_datos_yahoo as extrae
import solver.funciones_auxiliares as aux
import solver.line_search as line
import solver.modelo_markowitz as mkv
import solver.utils as utils
import solver.optimizacion_numerica as opt

In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [4]:
datos = extrae.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


In [5]:
datos.head()

,005930.KS,0688.HK,0700.HK,1398.HK,2010.SR,7203.T,AAPL,AI.PA,AMT,AMZN,BABA,BHP.AX,BRK-A,CCI,COP,CVX,DLR,FB,GOOG,HD,HON,JNJ,JPM,KO,LIN,LMT,MA,MC.PA,MRK,MSFT,NSRGY,PEP,PFE,PG,PLD,RDSA.AS,RELIANCE.NS,RHHBY,RIO.L,RTX,T,UNH,UNP,UPS,V,VOO,VTI,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,26600.0,24.704800,112.800003,5.77,79.50,7507.0,109.330002,89.786400,99.669998,308.519989,103.599998,27.603399,223600.0,79.510002,68.919998,112.580002,66.410004,78.449997,523.373108,103.430000,95.556229,104.519997,62.490002,42.139999,129.949997,193.309998,85.680000,130.850006,57.189999,46.759998,72.650002,94.440002,31.33,90.440002,43.430000,27.750000,442.774994,33.910000,2970.0,72.397736,33.869999,100.779999,118.610001,110.379997,66.254997,188.399994,105.919998,46.959999,85.900002,92.830002
2015-01-05,26660.0,24.951799,113.500000,5.80,79.50,7507.0,106.250000,87.005997,98.230003,302.190002,101.000000,27.547300,220980.0,79.000000,65.639999,108.080002,67.690002,77.190002,512.463013,101.260002,93.735291,103.790001,60.549999,42.139999,126.519997,189.289993,83.269997,127.050003,58.040001,46.330002,70.959999,93.730003,31.16,90.010002,43.400002,26.615000,437.924988,34.029999,2883.5,71.189430,33.549999,99.120003,114.599998,108.169998,64.792503,185.089996,104.099998,46.570000,85.650002,90.290001
2015-01-06,25900.0,24.605900,120.000000,5.71,77.00,7300.0,106.260002,86.279999,97.970001,295.290009,103.320000,26.267099,220450.0,78.849998,62.930000,108.029999,67.480003,76.150002,500.585632,100.949997,93.516014,103.279999,58.980000,42.459999,124.900002,188.399994,83.089996,125.599998,60.320000,45.650002,70.610001,93.019997,31.42,89.599998,43.549999,26.514999,418.049988,33.900002,2944.5,70.182503,33.599998,98.919998,112.230003,107.459999,64.375000,183.270004,103.080002,47.040001,86.309998,89.809998
2015-01-07,26140.0,24.507099,124.400002,5.75,78.25,7407.0,107.750000,86.669601,99.000000,298.420013,102.129997,26.267099,223480.0,80.500000,63.349998,107.940002,68.019997,76.150002,499.727997,104.410004,94.192909,105.559998,59.070000,42.990002,126.300003,190.830002,84.220001,125.699997,61.610001,46.230000,70.750000,95.739998,31.85,90.070000,44.209999,26.870001,427.149994,33.990002,2962.5,70.943993,33.169998,99.930000,112.849998,108.459999,65.237503,185.559998,104.309998,46.189999,88.599998,90.720001
2015-01-08,26280.0,23.864799,127.300003,5.72,79.25,7554.0,111.889999,90.317703,99.919998,300.459991,105.029999,26.519400,226680.0,81.769997,64.930000,110.410004,68.910004,78.180000,501.303680,106.720001,95.908974,106.389999,60.389999,43.509998,128.380005,195.130005,85.529999,129.649994,62.849998,47.590000,71.459999,97.480003,32.50,91.099998,44.220001,27.495001,421.024994,34.279999,3027.5,72.152298,33.500000,104.699997,117.080002,110.410004,66.112503,188.820007,106.150002,47.180000,90.470001,92.230003


In [0]:
mu = aux.calcular_rendimiento(datos)

In [0]:
S = aux.calcular_varianza(datos)

In [8]:
max(mu)

array(0.40221088)

In [0]:
rango =np.arange(0.4,1.1,0.1)

In [10]:
rango

array([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [0]:
res_sol1 = [mkv.markowitz(r,mu,S) for r in rango]

In [0]:
fo = lambda w: w@S@w
#w_ast = mkv.markowitz(r,mu,S)
n = mu.shape[0]
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#b = cp.array([r,1])
M = cp.ones((2,mu.shape[0]))
tol=1e-8
tol_backtracking=1e-14
#p_ast=fo(w_ast)

In [13]:
res_sol2 = [opt.Newtons_method_feasible_init_point(fo,
                                                   A,
                                                   utils.feasible_markowitz(r,mu),
                                                   tol,
                                                   tol_backtracking,
                                                   mkv.markowitz(r,mu,S),
                                                   fo(mkv.markowitz(r,mu,S)),
                                                   maxiter=50)[0] for r in rango]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.023	0.1226	4.9546	658.4719	---		1831.1723
1	0.023	0.0015	3.8955	7.8723	1	1831.1723
2	0.023	0.0	1.1442	0.0507	1	1831.1723
3	0.023	0.0	0.0262	0.0	1	1831.1723
Error of x with respect to x_ast: 0.026243401335918255
Approximate solution: [ 1.57795928e-01 -2.02691361e-02  1.56601997e-01 -9.18329212e-02
  3.19026593e-02  5.13753473e-02  8.69304413e-02  1.28914429e-02
  3.63069050e-02  2.84384982e-01 -5.23613218e-03  1.92382125e-03
  2.03108810e-01  9.41380756e-02  2.27258877e-02  1.48042182e-02
  8.15114814e-03  2.98341506e-02  5.79705178e-02  1.97287027e-01
  1.18301471e-01  1.27611328e-01  1.41376727e-01  1.36021595e-02
  8.77568968e-02  1.50457156e-01  1.67974885e-01  7.64946857e-02
  8.06725810e-02  8.28349470e-02  1.92517780e-01 -2.10512800e-02
  2.68907133e-02  7.70233469e-02  2.15614978e-02 -1.07745970e-01
  1.63347669e-01  1.72158174e-02  6.15006827e-02 -1.07599981e-01
 -5.03858517e-02  1.38527534e-01  1.0319657

La norma 2 está dada por

In [55]:
norm2 = np.zeros(7)
for i in range(7):
  norm2[i] = np.linalg.norm(res_sol1[i]-res_sol2[i], ord=2)
  print(np.linalg.norm(res_sol1[i]-res_sol2[i]))

0.06450578493633026
0.07434608399478973
0.03929472027534662
23.782816189824405
30.344634651869278
23.83392552330698
35.76872170331559


La norma 1 está dada por

In [57]:
norm1 = np.zeros(7)
for i in range(7):
  norm1[i] = np.linalg.norm(res_sol1[i]-res_sol2[i], ord=1)
  print(np.linalg.norm(res_sol1[i]-res_sol2[i], ord=1))

0.1102295806504027
0.12964678800100682
0.06660254290270241
40.5029000351
50.478340548902594
37.8330048982469
72.23070236034027


La restricción del rendimiento del primer solver es:

In [60]:
mu1 = np.zeros(7)
for i in range(7):
  mu1[i] = sum(res_sol1[i]*mu)
  print(sum(res_sol1[i]*mu))

0.4000000000000006
0.5000000000000006
0.6000000000000003
0.7000000000000005
0.8000000000000003
0.9000000000000001
0.9999999999999998


La restricción de rendimiento para el segundo solver es:

In [61]:
mu2 = np.zeros(7)
for i in range(7):
  mu2[i] = sum(res_sol2[i]*mu)
  print(sum(res_sol2[i]*mu))

0.3999999999999993
0.49999999999999856
0.5999999999999992
0.7000000000000005
0.7999999999999976
0.8999999999999967
0.9999999999999964


La restricción de unicidad para el primer solver es:

In [62]:
u1 = np.zeros(7)
for i in range(7):
  u1[i] = sum(res_sol1[i])
  print(sum(res_sol1[i]))

1.0000000000000004
0.9999999999999991
1.0
0.9999999999999989
1.0000000000000022
0.9999999999999996
1.000000000000001


La restricción de unicidad para el segundo solver es:

In [63]:
u2 = np.zeros(7)
for i in range(7):
  u2[i] = sum(res_sol2[i])
  print(sum(res_sol2[i]))

0.9999999999999996
0.999999999999999
1.000000000000003
0.9999999999999987
1.0000000000000049
0.9999999999999998
0.9999999999999908


La solución a la función objetivo para el primer solver es:

In [64]:
s1 = np.zeros(7)
for i in range(7):
  s1[i] = res_sol1[i]@S@res_sol1[i]
  print(res_sol1[i]@S@res_sol1[i])

9.397691724105942e-05
0.00012574768623457936
0.00016502392138388028
0.0002118056226889634
0.0002660927901498277
0.00032788542376647404
0.00039718352353889904


La solución a la funcióón objetivo para el segundo solver es:

In [65]:
s2 = np.zeros(7)
for i in range(7):
  s2[i] = res_sol2[i]@S@res_sol2[i]
  print(res_sol2[i]@S@res_sol2[i])

9.397988967700636e-05
0.00012575162219927977
0.00016502491199219758
0.0006680283861108914
0.0008424468977042823
0.0006757518970911845
0.0030961683015238864


In [0]:
tabla = pd.DataFrame({'||Dif||': norm2,
                     '|Dif|': norm1,
                     'w1*mu': mu1,
                     'w2*mu': mu2,
                     'Sigma1': s1,
                     'Sigma2': s2})

In [70]:
tabla

,||Dif||,|Dif|,w1*mu,w2*mu,Sigma1,Sigma2
0,0.064506,0.110230,0.4,0.4,0.000094,0.000094
1,0.074346,0.129647,0.5,0.5,0.000126,0.000126
2,0.039295,0.066603,0.6,0.6,0.000165,0.000165
3,23.782816,40.502900,0.7,0.7,0.000212,0.000668
4,30.344635,50.478341,0.8,0.8,0.000266,0.000842
5,23.833926,37.833005,0.9,0.9,0.000328,0.000676
6,35.768722,72.230702,1.0,1.0,0.000397,0.003096


In [67]:
mu2

array([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])